In [ ]:
# Start writing code here...They are not formally
defined in WordNet. Notice that the synset and its genus should belong
to the same grammatical category. This is not required for differentia. For
example, ricotta and its genus (cheese) in the following synset are nouns,
while the differentia contains two adjectives soft and Italian

Note su genus-differentia:

They are not formally
defined in WordNet. Notice that the synset and its genus should belong
to the same grammatical category. This is not required for differentia. For
example, ricotta and its genus (cheese) in the following synset are nouns,
while the differentia contains two adjectives soft and Italian

from http://eprints-phd.biblio.unitn.it/1265/1/phd-thesis.pdf

In [ ]:
import pandas as pd
from pathlib import Path
import nltk

import src.word_sense_disambiguation as wsd

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
concepts = pd.read_csv(Path("data/definitions.tsv"), sep="\t")

definitions = concepts['Concetto 1']

pos = []

for definition in definitions:
    pos.extend(nltk.pos_tag(nltk.word_tokenize(definition)))

# pos is a pair (lemma, pos type)
candidates = map(lambda pos: pos[0], filter(lambda pos: pos[1] in ['NN'], pos))

long_context = " ".join(definitions)

for candidate in set(candidates):
    found = nltk.wsd.lesk(wsd.bow_model(long_context),candidate,'n')
    print(f"{candidate}-{found}: {found.definition()}")

abstract-Synset('outline.n.02'): a sketchy summary of the main points of an argument or theory
compliance-Synset('complaisance.n.01'): a disposition or tendency to yield to the will of others
community-Synset('community.n.06'): (ecology) a group of interdependent organisms inhabiting the same region and interacting with each other
front-Synset('movement.n.04'): a group of people with a common ideology who try together to achieve certain general goals
principle-Synset('principle.n.01'): a basic generalization that is accepted as true and that can be used as a basis for reasoning or conduct
condition-Synset('condition.n.08'): the procedure that is varied in order to estimate a variable's effect by comparison with a control condition
entity-Synset('entity.n.01'): that which is perceived or known or inferred to have its own distinct existence (living or nonliving)
concept-Synset('concept.n.01'): an abstract or general idea inferred or derived from specific instances
idea-Synset('theme.n.03

In [ ]:
#def build_concept_bow(definitions):
bow = set()

for definition in definitions:
    bow = bow.union(wsd.bow_model(definition)) 


    

In [ ]:
bow

{'.',
 'To',
 'Usually',
 'a',
 'abstract',
 'administrated',
 'agree',
 'all',
 'allows',
 'and',
 'application',
 'based',
 'being',
 'by',
 'code',
 'community',
 'compliance',
 'concept',
 'condition',
 'correct',
 'courts',
 'determining',
 'entity',
 'equality',
 'fair',
 'fairness',
 'for',
 'front',
 'idea',
 'in',
 'involved',
 'is',
 'just',
 'laws',
 'legal',
 'moral',
 'morally',
 'most',
 'of',
 'on',
 'other',
 'parts',
 'people',
 'points',
 'principle',
 'public',
 'quality',
 'reasonable',
 'recognition',
 'refers',
 'respect',
 'right',
 'rights',
 'rules',
 'shared',
 'that',
 'the',
 'their',
 'to',
 'view',
 'what',
 'when',
 'with'}

In [ ]:
a = set()
a)

{1, 2, 3}

In [ ]:
from nltk.corpus import wordnet as wn

wn.synsets('fairness', 'n')[0].definition()

'conformity with rules or standards'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=afb22156-bb61-4d65-847d-18db79c0d4d2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>